# Import required packages

In [1]:
# Optimization Packages
using JuMP
using SCS
using GLPK

In [2]:
# Linear Algebra packages
using LinearAlgebra
using SparseArrays

# Part 2
Julia is numerical computation language used for many scientific applications. Here we will look how to use this language to solve optimization problems. 

## Part 2.1 Creating a Model
Models are created with the `Model()` function. The `with_optimizer` syntax is used to specify the optimizer to be used:

In [ ]:
? Model

In [4]:
model = Model(with_optimizer(GLPK.Optimizer))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

## Part 2.2 Declaring Variables
The following commands will create two variables, `x` and `y`, with both lower and upper bounds. Note the first argument is our `model` model. These variables (`x` and `y`) are associated with this model and cannot be used in another model.

Also go to the link for nice description on how to create JuMP variables [link](http://www.juliaopt.org/JuMP.jl/v0.19.0/variables/#Variables-1).

In [11]:
? @variable

```
@variable(model, kw_args...)
```

Add an *anonymous* variable to the model `model` described by the keyword arguments `kw_args` and returns the variable.

```
@variable(model, expr, args..., kw_args...)
```

Add a variable to the model `model` described by the expression `expr`, the positional arguments `args` and the keyword arguments `kw_args`. The expression `expr` can either be (note that in the following the symbol `<=` can be used instead of `≤` and the symbol `>=`can be used instead of `≥`)

  * of the form `varexpr` creating variables described by `varexpr`;
  * of the form `varexpr ≤ ub` (resp. `varexpr ≥ lb`) creating variables described by `varexpr` with upper bounds given by `ub` (resp. lower bounds given by `lb`);
  * of the form `varexpr == value` creating variables described by `varexpr` with fixed values given by `value`; or
  * of the form `lb ≤ varexpr ≤ ub` or `ub ≥ varexpr ≥ lb` creating variables described by `varexpr` with lower bounds given by `lb` and upper bounds given by `ub`.

The expression `varexpr` can either be

  * of the form `varname` creating a scalar real variable of name `varname`;
  * of the form `varname[...]` or `[...]` creating a container of variables (see [Containers in macros](@ref)).

The recognized positional arguments in `args` are the following:

  * `Bin`: Sets the variable to be binary, i.e. either 0 or 1.
  * `Int`: Sets the variable to be integer, i.e. one of ..., -2, -1, 0, 1, 2, ...
  * `Symmetric`: Only available when creating a square matrix of variables, i.e. when `varexpr` is of the form `varname[1:n,1:n]` or `varname[i=1:n,j=1:n]`. It creates a symmetric matrix of variable, that is, it only creates a new variable for `varname[i,j]` with `i ≤ j` and sets `varname[j,i]` to the same variable as `varname[i,j]`.
  * `PSD`: The square matrix of variable is both `Symmetric` and constrained to be positive semidefinite.

The recognized keyword arguments in `kw_args` are the following:

  * `base_name`: Sets the name prefix used to generate variable names. It corresponds to the variable name for scalar variable, otherwise, the variable names are set to `base_name[...]` for each index `...` of the axes `axes`.
  * `lower_bound`: Sets the value of the variable lower bound.
  * `upper_bound`: Sets the value of the variable upper bound.
  * `start`: Sets the variable starting value used as initial guess in optimization.
  * `binary`: Sets whether the variable is binary or not.
  * `integer`: Sets whether the variable is integer or not.
  * `variable_type`: See the "Note for extending the variable macro" section below.
  * `container`: Specify the container type, see [Containers in macros](@ref).

## Examples

The following are equivalent ways of creating a variable `x` of name `x` with lower bound 0:

```julia
# Specify everything in `expr`
@variable(model, x >= 0)
# Specify the lower bound using a keyword argument
@variable(model, x, lower_bound=0)
# Specify everything in `kw_args`
x = @variable(model, base_name="x", lower_bound=0)
```

The following are equivalent ways of creating a `DenseAxisArray` of index set `[:a, :b]` and with respective upper bounds 2 and 3 and names `x[a]` and `x[b]`.

```julia
ub = Dict(:a => 2, :b => 3)
# Specify everything in `expr`
@variable(model, x[i=keys(ub)] <= ub[i])
# Specify the upper bound using a keyword argument
@variable(model, x[i=keys(ub)], upper_bound=ub[i])
```

## Note for extending the variable macro

The single scalar variable or each scalar variable of the container are created using `add_variable(model, build_variable(_error, info, extra_args...; extra_kw_args...))` where

  * `model` is the model passed to the `@variable` macro;
  * `_error` is an error function with a single `String` argument showing the `@variable` call in addition to the error message given as argument;
  * `info` is the `VariableInfo` struct containing the information gathered in `expr`, the recognized keyword arguments (except `base_name` and `variable_type`) and the recognized positional arguments (except `Symmetric` and `PSD`);
  * `extra_args` are the unrecognized positional arguments of `args` plus the value of the `variable_type` keyword argument if present. The `variable_type` keyword argument allows the user to pass a position argument to `build_variable` without the need to give a positional argument to `@variable`. In particular, this allows the user to give a positional argument to the `build_variable` call when using the anonymous single variable syntax `@variable(model, kw_args...)`; and
  * `extra_kw_args` are the unrecognized keyword argument of `kw_args`.

## Examples

The following creates a variable `x` of name `x` with `lower_bound` 0 as with the first example above but does it without using the `@variable` macro

```julia
info = VariableInfo(true, 0, false, NaN, false, NaN, false, NaN, false, false)
JuMP.add_variable(model, JuMP.build_variable(error, info), "x")
```

The following creates a `DenseAxisArray` of index set `[:a, :b]` and with respective upper bounds 2 and 3 and names `x[a]` and `x[b]` as with the second example above but does it without using the `@variable` macro

```julia
# Without the `@variable` macro
data = Vector{JuMP.variable_type(model)}(undef, length(keys(ub)))
x = JuMP.Containers.DenseAxisArray(data, keys(ub))
for i in keys(ub)
    info = VariableInfo(false, NaN, true, ub[i], false, NaN, false, NaN, false, false)
    x[i] = JuMP.add_variable(model, JuMP.build_variable(error, info), "x[$i]")
end
```

The following are equivalent ways of creating a `Matrix` of size `N x N` with variables custom variables created with a JuMP extension using the `Poly(X)` positional argument to specify its variables:

```julia
# Using the `@variable` macro
@variable(model, x[1:N,1:N], Symmetric, Poly(X))
# Without the `@variable` macro
x = Matrix{JuMP.variable_type(model, Poly(X))}(N, N)
info = VariableInfo(false, NaN, false, NaN, false, NaN, false, NaN, false, false)
for i in 1:N, j in i:N
    x[i,j] = x[j,i] = JuMP.add_variable(model, build_variable(error, info, Poly(X)), "x[$i,$j]")
end
```


In [5]:
@variable(model, 0 <= x <= 2)
y = @variable(model, base_name="y", lower_bound=0, upper_bound=30)

Decision Varible

In [ ]:
? lower_bound

In [ ]:
? upper_bound

In [ ]:
println("Lower bound on x: ", lower_bound(x))
println("Upper bound on y: ", upper_bound(y))

## Part 2.3 Setting the Objective
Next we'll set our objective on the `model` we defined earlier. The objective sense, `Max` or `Min`, should be provided as the second argument. 

Note also that we don't have a multiplication `*` symbol between `5` and our variable `x` - Julia is smart enough to not need it. We can also use `*`. Choose between the two convention based on convienence.

In [ ]:
? @objective

In [6]:
@objective(model, Max, 5x + 3 * y)

5 x + 3 Decision Varible

## Part 2.4 Setting the Constraints
Adding constraints is a lot like setting the objective. Here we create a less-than-or-equal-to constraint using `<=`, but we can also create equality constraints using `==` and greater-than-or-equal-to constraints with `>=`.

In [8]:
? @constraints

```
@constraints(m, args...)
```

adds groups of constraints at once, in the same fashion as @constraint. The model must be the first argument, and multiple constraints can be added on multiple lines wrapped in a `begin ... end` block. For example:

```
@constraints(m, begin
  x >= 1
  y - w <= 2
  sum_to_one[i=1:3], z[i] + y == 1
end)
```


In [9]:
? @constraint

```
@constraint(m::Model, expr)
```

Add a constraint described by the expression `expr`.

```
@constraint(m::Model, ref[i=..., j=..., ...], expr)
```

Add a group of constraints described by the expression `expr` parametrized by `i`, `j`, ...

The expression `expr` can either be

  * of the form `func in set` constraining the function `func` to belong to the set `set` which is either a [`MathOptInterface.AbstractSet`](http://www.juliaopt.org/MathOptInterface.jl/v0.6.2/apireference.html#Sets-1) or one of the JuMP shortcuts [`SecondOrderCone`](@ref), [`RotatedSecondOrderCone`](@ref) and [`PSDCone`](@ref), e.g. `@constraint(model, [1, x-1, y-2] in SecondOrderCone())` constrains the norm of `[x-1, y-2]` be less than 1;
  * of the form `a sign b`, where `sign` is one of `==`, `≥`, `>=`, `≤` and `<=` building the single constraint enforcing the comparison to hold for the expression `a` and `b`, e.g. `@constraint(m, x^2 + y^2 == 1)` constrains `x` and `y` to lie on the unit circle;
  * of the form `a ≤ b ≤ c` or `a ≥ b ≥ c` (where `≤` and `<=` (resp. `≥` and `>=`) can be used interchangeably) constraining the paired the expression `b` to lie between `a` and `c`;
  * of the forms `@constraint(m, a .sign b)` or `@constraint(m, a .sign b .sign c)` which broadcast the constraint creation to each element of the vectors.

## Note for extending the constraint macro

Each constraint will be created using `add_constraint(m, build_constraint(_error, func, set))` where

  * `_error` is an error function showing the constraint call in addition to the error message given as argument,
  * `func` is the expression that is constrained
  * and `set` is the set in which it is constrained to belong.

For `expr` of the first type (i.e. `@constraint(m, func in set)`), `func` and `set` are passed unchanged to `build_constraint` but for the other types, they are determined from the expressions and signs. For instance, `@constraint(m, x^2 + y^2 == 1)` is transformed into `add_constraint(m, build_constraint(_error, x^2 + y^2, MOI.EqualTo(1.0)))`.

To extend JuMP to accept new constraints of this form, it is necessary to add the corresponding methods to `build_constraint`. Note that this will likely mean that either `func` or `set` will be some custom type, rather than e.g. a `Symbol`, since we will likely want to dispatch on the type of the function or set appearing in the constraint.


In [12]:
# We have named the constarint as "con" 
@constraint(model, con, 1x + 5y <= 3)

con : x + 5 Decision Varible <= 3.0

## Part 2.5 Optimizing the objective
Models are solved with the `JuMP.optimize!` function. After the call to `JuMP.optimize!` has finished, we need to query what happened. The solve could terminate for a number of reasons. First, the solver might have found the optimal solution or proved that the problem is infeasible. However, it might also have run into numerical difficulties or terminated due to a setting such as a time limit. We can ask the solver why it stopped using the `JuMP.termination_status` function.

In [13]:
? optimize! 

search: optimize! optimize_ticks optimizer_index OptimizerFactory



```
optimize!(model::Model,
          optimizer_factory::Union{Nothing, OptimizerFactory}=nothing;
          bridge_constraints::Bool=true,
          ignore_optimize_hook=(model.optimize_hook === nothing),
          kwargs...)
```

Optimize the model. If `optimizer_factory` is not `nothing`, it first sets the optimizer to a new one created using the optimizer factory. The factory can be created using the [`with_optimizer`](@ref) function. If `optimizer_factory` is `nothing` and no optimizer was set to `model` before calling this function, a [`NoOptimizer`](@ref) error is thrown.

Keyword arguments `kwargs` are passed to the `optimize_hook`. An error is thrown if `optimize_hook` is `nothing` and keyword arguments are provided.

## Examples

The optimizer factory can either be given in the [`Model`](@ref) constructor as follows:

```julia
model = Model(with_optimizer(GLPK.Optimizer))
# ...fill model with variables, constraints and objectives...
# Solve the model with GLPK
optimize!(model)
```

or in the `optimize!` call as follows:

```julia
model = Model()
# ...fill model with variables, constraints and objectives...
# Solve the model with GLPK
optimize!(model, with_optimizer(GLPK.Optimizer))
```


In [14]:
optimize!(model)

In [15]:
? termination_status()

```
termination_status(model::Model)
```

Return the reason why the solver stopped (i.e., the MathOptInterface model attribute `TerminationStatus`).


In [16]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

## Part 2.6 Results
As the solver found an optimal solution, we expect the solution returned to be a primal-dual pair of feasible solutions with zero duality gap. We can verify the primal and dual status as follows to confirm this:

In [17]:
? primal_status()

```
primal_status(model::Model)
```

Return the status of the most recent primal solution of the solver (i.e., the MathOptInterface model attribute `PrimalStatus`).


In [19]:
? dual_status()

```
dual_status(model::Model)
```

Return the status of the most recent dual solution of the solver (i.e., the MathOptInterface model attribute `DualStatus`).


In [23]:
primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

In [27]:
println(dual_status(model))

FEASIBLE_POINT


Note that the primal and dual status only inform that the primal and dual solutions are feasible and it is only because we verified that the termination status is `OPTIMAL` that we can conclude that they form an optimal solution.

Now we will see:
- How to query the objective value?
- How to query the primal result values of the `x` and `y` variables?

In [34]:
? objective_value()

```
objective_value(model::Model)
```

Return the objective value after a call to `optimize!(model)`.


In [35]:
? value()

```
value(con_ref::ConstraintRef)
```

Get the primal value of this constraint in the result returned by a solver. That is, if `con_ref` is the reference of a constraint `func`-in-`set`, it returns the value of `func` evaluated at the value of the variables (given by [`value(::VariableRef)`](@ref)). Use [`has_values`](@ref) to check if a result exists before asking for values.

## Note

For scalar contraints, the constant is moved to the `set` so it is not taken into account in the primal value of the constraint. For instance, the constraint `@constraint(model, 2x + 3y + 1 == 5)` is transformed into `2x + 3y`-in-`MOI.EqualTo(4)` so the value returned by this function is the evaluation of `2x + 3y`. ```

---

```
value(v::VariableRef)
```

Get the value of this variable in the result returned by a solver. Use [`has_values`](@ref) to check if a result exists before asking for values.

---

```
value(ex::GenericAffExpr, var_value::Function)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(v::GenericAffExpr)
```

Evaluate an `GenericAffExpr` given the result returned by a solver. Replaces `getvalue` for most use cases.

---

```
value(p::NonlinearParameter)
```

Return the current value stored in the nonlinear parameter `p`.

# Example

```jldoctest
model = Model()
@NLparameter(model, p == 10)
value(p)

# output
10.0
```

---

```
value(ex::NonlinearExpression, var_value::Function)
```

Evaluate `ex` using `var_value(v)` as the value for each variable `v`.

---

```
value(ex::NonlinearExpression)
```

Evaluate `ex` using `value` as the value for each variable `v`.


In [33]:
println("Max value of objective function: ", objective_value(model))
println("Value of the variable x: ", value(x))
println("Value of the variable y: ", value(y))

Max value of objective function: 10.6
Value of the variable x: 2.0
Value of the variable y: 0.2


## Part 2.7 Duality
We will now some simple syntax to access dual information of the primal LP. We can the value of the dual variable associated with the constraint `con`, which we bound to a Julia variable when defining the constraint, for more info refer this [link](http://www.juliaopt.org/JuMP.jl/v0.19.0/constraints/#constraint_duality-1).

In [1]:
? has_dual()

No documentation found.

Binding `has_dual` does not exist.


In [36]:
? dual()

```
dual(con_ref::ConstraintRef)
```

Get the dual value of this constraint in the result returned by a solver. Use `has_dual` to check if a result exists before asking for values. See also [`shadow_price`](@ref).


In [37]:
dual(con)

-0.6

In [42]:
# To query the dual variables associated with the variable bounds, things are a little trickier 
# as we first need to obtain a reference to the constraint:
x_upper = UpperBoundRef(x)
println("Reference to the upper bound constraint on varible x: ", x_upper)
println(dual(x_upper))

x <= 2.0
-4.4


In [43]:
y_lower = LowerBoundRef(y)
println(y_lower)
println(dual(y_lower))

Decision Varible >= 0.0
0.0
